# Bring your own Model - IRIS classifier example.

This notebook can be compared to [tensorflow_iris_dnn_classifier_using_estimators.ipynb](https://github.com/awslabs/sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_iris_dnn_classifier_using_estimators/tensorflow_iris_dnn_classifier_using_estimators.ipynb). It seeks to solve the same problem, but instead of training by using Amazon SageMaker's distributed, managed training functionality, it relies on the user to train locally or bring a pre-trained model, and then setup a real-time hosted endpoint in SageMaker. To do that, we'll rely on the same set of functions for training. 

Consider the following mdoel definition for IRIS classification. This mdoe uses the ``tensorflow.estimator.DNNClassifier`` which is a pre-defined enstimator module for its model definition. 

In [ ]:
role = <<Your_Sagemaker_Role>>

In [ ]:
import os
import numpy as np
import tensorflow as tf

INPUT_TENSOR_NAME = 'x'


def estimator_fn(run_config, params):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=3,
                                      config=run_config)


def serving_input_fn():
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()


def train_input_fn(training_dir, params):
    """Returns input function that would feed the model during training"""
    return _generate_input_fn(training_dir, 'iris_training.csv')


def eval_input_fn(training_dir, params):
    """Returns input function that would feed the model during evaluation"""
    return _generate_input_fn(training_dir, 'iris_test.csv')


def _generate_input_fn(training_dir, training_filename):
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=os.path.join(training_dir, training_filename),
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)

Create an estimator object with this model definition.

In [ ]:
classifier = estimator_fn(run_config = None, params = None)

Download and make the iris dataset from TensorFlow's repository.

In [ ]:
import os 
from six.moves.urllib.request import urlopen

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

Create the input streamer object.

In [ ]:
train_func = train_input_fn('.', params = None)

Train using TensorFlow's ``tensorflow.Estimator.train`` method. The model is trained locally in the box.

In [ ]:
classifier.train(input_fn = train_func, steps = 1000)

There is a small difference between a SageMaker model and a TensorFlow model. The conversion is easy and fairly trivial. Simply move the tensorflow exported model into a directory ``export\Servo\`` and tar the entire directory. SageMaker will recognize this as a loadable TensorFlow model.

In [ ]:
exported_model = classifier.export_savedmodel(export_dir_base = 'export/Servo/', 
                               serving_input_receiver_fn = serving_input_fn)
print (exported_model)
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

Open a new sagemaker session and upload the model into the default S3 bucket under the directory ``model``.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

Use the ``sagemaker.tensorflow.model.TensorflowModel`` class directly to setup a trained model in a sagemaker session.

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  entry_point = 'iris_dnn_classifier.py')

Deploy the newly created SageMaker model to an endpoint.

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                          instance_type='ml.c4.xlarge')

Run a sample prediction on a sample to ensure that it works. Expect result ``1`` for this particular sample.

In [ ]:
predict_samples = {}
predict_samples ={INPUT_TENSOR_NAME: np.array([[6.4,3.2,4.5,1.5]])}
predictor.predict(np.array([[6.4,3.2,4.5,1.5]]))

Delete all temporary directories so that we are not affecting the next run.

In [ ]:
os.remove('model.tar.gz')
import shutil
shutil.rmtree('export')